In [ ]:
import re, math, unicodedata
from typing import Optional
import numpy as np
import pandas as pd
from google.colab import drive

## Extracción
En esta etapa se carga el dataset original en formato CSV.  
Se verifica la estructura general, tipos de datos, valores faltantes y posibles inconsistencias.

In [ ]:
drive.mount('/content/drive')

URL_DATASET_RAW = "/content/drive/MyDrive/Alan/dataset/raw/train_dirty.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datos = pd.read_csv(URL_DATASET_RAW, low_memory=False)
print("Shape:", datos.shape)
print("Columnas:", len(datos.columns))
datos.head(3)

Shape: (2000, 22)
Columnas: 22


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range,Unnamed: 21
0,842,0,2.2,0,1,0,7168 MB,0.6,188,2,...,756,2549MB,9,7,19,0,0,1,"$42,000 MXN",NaN
1,1021,1,0.5,1,0,1,53 GB,0.7,136,3,...,1988,2631MB,17,3,7,1,1,no,"$49,089",NaN
2,563,1,0.5,1,2,1,41984 MB,0.9,145,5,...,1716,2603MB,11,2,9,1,1,0,"USD 16,049",NaN


### Quitar columna vacia

In [ ]:
datos.drop(columns=["Unnamed: 21"], inplace=True)
datos.head(3)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7168 MB,0.6,188,2,...,20,756,2549MB,9,7,19,0,0,1,"$42,000 MXN"
1,1021,1,0.5,1,0,1,53 GB,0.7,136,3,...,905,1988,2631MB,17,3,7,1,1,no,"$49,089"
2,563,1,0.5,1,2,1,41984 MB,0.9,145,5,...,1263,1716,2603MB,11,2,9,1,1,0,"USD 16,049"


## Transformación
En esta etapa se realiza la limpieza y normalización de los datos.

Pasar MB a GB en memoria y tambien pesos y euros a dolares / 1000.

In [ ]:
def strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))

def normalizar_texto(x: Optional[str]) -> Optional[str]:
    if pd.isna(x): return x
    s = str(x).strip()
    s = strip_accents(s)
    s = s.replace("\n"," ").replace("\t"," ")
    s = re.sub(r"\s+", " ", s)
    return s

Verdadero = {"1","yes", "Yes"}
Falso = {"0","no", "No"}

def parsear_bool(x):
    if pd.isna(x): return np.nan
    s = normalizar_texto(str(x)).lower()
    if s in Verdadero:  return 1
    if s in Falso: return 0
    try:
        v = float(re.sub(r"[^0-9\.\-+eE]", "", s))
        return 1 if not math.isnan(v) and v >= 0.5 else 0
    except Exception:
        return np.nan

Tasas = {
    "usd": 1.0,
    "mxn": 0.054,
    "eur": 1.16,
}

def extraer_primer_float(texto: str):
    if not texto:
        return np.nan
    for parte in texto.split():
        try:
            return float(parte)
        except ValueError:
            continue
    return np.nan

def parsear_numeros(s: Optional[str]):
    if s is None or (isinstance(s, float) and math.isnan(s)): return np.nan
    orig = str(s).strip().lower()
    if orig in {"", "na", "n/a", "null", "none", "nan", "-", "--", "s/d", "sd"}: return np.nan

    moneda = "usd"
    for key in Tasas.keys():
        if key in orig:
            moneda = key
            break

    if any(word in orig for word in ["peso", "mxn", "$"]) and "usd" not in orig:
        moneda = "mxn"
    if "€" in orig or "eur" in orig:
        moneda = "eur"

    s2 = (orig.replace("MXN","").replace("USD","").replace("EUR","")
              .replace("pesos","").replace("peso","")
              .replace("dolares","").replace("dólares","").replace("euros","")
              .replace("$","").replace("€","").replace(" gb", "000").replace("gb", "000")
              .replace(",",""))
    s2 = s2.replace("mil", "k")

    partes = re.split(r"\s*(?:-|a|to|–|—|/)\s*", s2)
    if len(partes) == 2 and all(re.search(r"\d", p) for p in partes):
        try:
            v1 = parsear_numeros(partes[0])
            v2 = parsear_numeros(partes[1])
            if pd.notna(v1) and pd.notna(v2):
                return (v1 + v2) / 2
        except:
            pass

    m = re.match(r"^([-+]?\d*\.?\d+)\s*[kK]$", s2)
    if m:
        return float(m.group(1)) * 1_000 * Tasas.get(moneda, 1)
    m = re.match(r"^([-+]?\d*\.?\d+)\s*[mM]$", s2)
    if m:
        return float(m.group(1)) * 1_000_000 * Tasas.get(moneda, 1)

    es_precio_memoria = any(tok in orig for tok in ["$", "€", "mxn", "usd", "eur", "peso", "pesos", "dolar", "dólar", "dolares", "dólares", "euros", "gb", " gb", "mb", " mb"])

    s3 = re.sub(r"[^0-9eE\.\-+]", " ", s2)
    s3 = re.sub(r"\s+", " ", s3).strip()
    valor = extraer_primer_float(s3)

    if pd.isna(valor):
        return np.nan

    if es_precio_memoria:
        valor_usd = valor * Tasas.get(moneda, 1)
        return float(valor_usd // 1000)
    else:
        return valor

### Ver el tipo de columnas dependiedo de los datos

In [ ]:
forzar_numero = set()
forzar_bool    = set()
forzar_texto    = set()

cols_numericas, cols_bool, cols_texto = [], [], []

for c in datos.columns:
    s = datos[c]
    if c in forzar_numero: cols_numericas.append(c); continue
    if c in forzar_bool: cols_bool.append(c);    continue
    if c in forzar_texto: cols_texto.append(c);    continue

    nun = s.dropna().nunique()
    if nun <= 6:
        cols_bool.append(c); continue

    numero_parseado = s.map(parsear_numeros)
    radio_num = numero_parseado.notna().mean()
    if radio_num >= 0.8:
        cols_numericas.append(c); continue

    cols_texto.append(c)

print("Numéricas:", cols_numericas[:20])
print("Booleanas:", cols_bool[:20])
print("Texto/Categóricas:", cols_texto[:20])

Numéricas: ['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'price_range']
Booleanas: ['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']
Texto/Categóricas: []


### Limpieza

In [ ]:
dataset_limpio = datos.copy()

for c in cols_texto:
    dataset_limpio[c] = dataset_limpio[c].map(normalizar_texto).str.lower()

for c in cols_bool:
    dataset_limpio[c] = dataset_limpio[c].map(parsear_bool)
    modo = dataset_limpio[c].mode()
    llenar = int(modo.iloc[0]) if not modo.empty else 0
    dataset_limpio[c] = dataset_limpio[c].fillna(llenar).astype(int)

for c in cols_numericas:
    num = dataset_limpio[c].map(parsear_numeros).astype(float)
    med = np.nanmedian(num)
    num = num.fillna(med)
    q1, q3 = np.percentile(num, [25, 75])
    iqr = q3 - q1
    lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr
    num = num.fillna(np.nanmedian(num))
    dataset_limpio[c] = num

In [ ]:
dataset_limpio.head(10)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842.0,0,2.2,0,1.0,0,7.0,0.6,188.0,2.0,...,20.0,756.0,2.0,9.0,7.0,19.0,0,0,1,2.0
1,1021.0,1,0.5,1,0.0,1,53.0,0.7,136.0,3.0,...,905.0,1988.0,2.0,17.0,3.0,7.0,1,1,0,2.0
2,563.0,1,0.5,1,2.0,1,41.0,0.9,145.0,5.0,...,1263.0,1716.0,2.0,11.0,2.0,9.0,1,1,0,16.0
3,615.0,1,2.5,0,0.0,0,10.0,0.8,131.0,6.0,...,1216.0,1786.0,2.0,16.0,8.0,11.0,1,0,0,2.0
4,1821.0,1,1.2,0,13.0,1,44.0,0.6,141.0,2.0,...,1208.0,1212.0,1.0,8.0,2.0,15.0,1,1,0,1.0
5,1859.0,0,0.5,1,3.0,0,22.0,0.7,164.0,1.0,...,1004.0,1654.0,1.0,17.0,1.0,10.0,1,0,0,0.0
6,1821.0,0,1.7,0,4.0,1,10.0,0.8,139.0,8.0,...,381.0,1018.0,3.0,13.0,8.0,18.0,1,0,1,1.0
7,1954.0,0,0.5,1,0.0,0,0.0,0.8,187.0,4.0,...,512.0,1149.0,0.0,16.0,3.0,5.0,1,1,1,0.0
8,1445.0,1,0.5,0,0.0,0,53.0,0.7,174.0,7.0,...,386.0,836.0,1.0,17.0,1.0,20.0,1,0,0,1.0
9,509.0,1,0.6,1,2.0,1,9.0,0.1,93.0,5.0,...,1137.0,1224.0,0.0,19.0,10.0,12.0,1,0,0,2.0


### Limpieza de filas que tienen valor 0 en precio o en almacenamiento

In [ ]:
dataset_limpio = dataset_limpio[(dataset_limpio["int_memory"] != 0) & (dataset_limpio["price_range"] != 0)]

dataset_limpio.head(10)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842.0,0,2.2,0,1.0,0,7.0,0.6,188.0,2.0,...,20.0,756.0,2.0,9.0,7.0,19.0,0,0,1,2.0
1,1021.0,1,0.5,1,0.0,1,53.0,0.7,136.0,3.0,...,905.0,1988.0,2.0,17.0,3.0,7.0,1,1,0,2.0
2,563.0,1,0.5,1,2.0,1,41.0,0.9,145.0,5.0,...,1263.0,1716.0,2.0,11.0,2.0,9.0,1,1,0,16.0
3,615.0,1,2.5,0,0.0,0,10.0,0.8,131.0,6.0,...,1216.0,1786.0,2.0,16.0,8.0,11.0,1,0,0,2.0
4,1821.0,1,1.2,0,13.0,1,44.0,0.6,141.0,2.0,...,1208.0,1212.0,1.0,8.0,2.0,15.0,1,1,0,1.0
6,1821.0,0,1.7,0,4.0,1,10.0,0.8,139.0,8.0,...,381.0,1018.0,3.0,13.0,8.0,18.0,1,0,1,1.0
8,1445.0,1,0.5,0,0.0,0,53.0,0.7,174.0,7.0,...,386.0,836.0,1.0,17.0,1.0,20.0,1,0,0,1.0
9,509.0,1,0.6,1,2.0,1,9.0,0.1,93.0,5.0,...,1137.0,1224.0,0.0,19.0,10.0,12.0,1,0,0,2.0
10,769.0,1,2.9,1,0.0,0,9.0,0.1,182.0,5.0,...,248.0,874.0,3.0,5.0,2.0,7.0,0,0,0,1.0
11,1520.0,1,2.2,0,5.0,1,33.0,0.5,177.0,8.0,...,151.0,1005.0,3.0,14.0,9.0,13.0,1,1,1,15.0


## Carga
En esta etapa se exporta el dataset limpio a un nuevo archivo CSV.  

In [ ]:
URL_DATASET_LIMPIO = "/content/drive/MyDrive/Alan/dataset/data/dataset.csv"

dataset_limpio.to_csv(URL_DATASET_LIMPIO, index=False)
print("Archivo guardado en:", URL_DATASET_LIMPIO)

Archivo guardado en: /content/drive/MyDrive/Alan/dataset/data/dataset.csv
